In [1]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent, AgentType, create_openai_functions_agent, AgentExecutor
from langchain_experimental.tools import PythonREPLTool
from langchain import hub
from constants import DB_HOST, DB_PORT, DB_USER, DB_PASS, DB_NAME, OPENAI_API_KEY, OPENAI_LLM_MODEL

In [2]:
assert DB_HOST
assert DB_PORT
assert DB_USER
assert DB_PASS
assert DB_NAME
assert OPENAI_API_KEY
assert OPENAI_LLM_MODEL

In [11]:
# Step 1: Set up database connection with SQLAlchemy
def initialize_db_connection():
    """Initialize SQLAlchemy engine and session for MySQL database."""
    try:
        engine = create_engine(
            f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}",
            echo=True, # Verbose logging for debugging
            pool_pre_ping=True # Ensure active connections
        )
        Session = sessionmaker(bind=engine)
        session = Session()
        print("Database connection established!")
        return engine, session
    except Exception as e:
        print(f"Failed to connect to database: {e}")
        return None, None

# Step 2: Configure LangChain SQLDatabase
def get_sql_database(engine):
    """Wrap SQLAlchemy engine in LangChain's SQLDatabase."""
    try:
        db = SQLDatabase(engine)
        print("LangChain SQLDatabase initialized!")
        return db
    except Exception as e:
        print(f"Failed to initialize SQLDatabase: {e}")
        return None

# Step 3: Initialize LLM
def initialize_llm():
    """Set up OpenAI LLM with API key"""
    try:
        llm = ChatOpenAI(
            model=OPENAI_LLM_MODEL,
            api_key=OPENAI_API_KEY,
            temperature=0
        )
        print("OpenAI LLM initialized!")
        return llm
    except Exception as e:
        print(f"Failed to initialize LLM: {e}")
        return None

# Step 4: Get DB Toolkit
def get_db_tool(db, llm):
    """Get SQLDatabaseToolkit"""
    try:
        toolkit = SQLDatabaseToolkit(db=db, llm=llm)
        print("DB Tool initialized!")
        return toolkit
    except Exception as e:
        print("failed to get db tool: {e}")
        return None
    

# Step 5: Create PythonREPLTool
def get_repl_tool():
    """Instantiate a PythonREPLTool"""
    try:
        repl = PythonREPLTool()
        print("REPL Tool initialized!")
        return repl
    except Exception as e:
        print(f"Failed to get REPL Tool: {e}")
        return None

# Step 6: Generate SQL+Viz Prompt
def get_prompt():
    """Set up a prompt instructing the agent to generate SQL then Plotly code."""
    try:
        instructions = """You are an agent designed to query a MySQL database and generate Python code for plotting data using Plotly.
        Input will be a natural language query (e.g., 'Plot monthly sales as a line chart').
        1. Generate a SQL query to fetch the required data from the database.
        2. Use the SQL results to create Plotly code (not Matplotlib) and return it in ```python <code>``` format.
        If the query is unclear or data is unavailable, return 'Insufficient data for visualization'.
        Assume the database has tables like 'orders' (for SalesTrend) or 'transactions' (for FinQuery)."""
        prompt = hub.pull("langchain-ai/openai-functions-template").partial(instructions=instructions)
        print("Prompt initialized!")
        return prompt
    except Exception as e:
        print(f"Failed to generate prompt: {e}")
        return None

    
# Step 7: Create SQL+Viz Agent
def create_agent(db_tool, repl_tool, llm, prompt):
    """Build SQL+Viz agent with LangChain toolkit."""
    try:
        tools = [db_tool, repl_tool]
        agent = create_openai_functions_agent(llm, tools, prompt)
        print("Agent initialized!")
        return agent
    except Exception as e:
        print(f"Failed to create agent: {e}")
        return None

# Step 8: Build executor (main function)
def create_agent_executor():
    """Build an agent executor for SQL+Viz capabilities"""
    engine, session = initialize_db_connection()
    if engine and session:
        db = get_sql_database(engine)
        if db:
            llm = initialize_llm()
            if llm:
                db_tool = get_db_tool(db, llm)
                if db_tool:
                    repl_tool = get_repl_tool()
                    if repl_tool:
                        prompt = get_prompt()
                        if prompt:
                            agent = create_agent(db_tool, repl_tool, llm, prompt)
                            if agent:
                                try:
                                    tools = [db_tool, repl_tool]
                                    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
                                    return executor, agent
                                except:
                                    print(f"Failed to create Executor: {e}")
                                    return None, None
    return None, None

# Step 9: Process Visualization Query’s 
def run_visualization(agent, query, session):
    """Execute natural language visualization query, query the database, and return formatted response."""
    try:
        response = agent.invoke({"input": query})["output"]
        # Format the response
        if "Insufficient data" in response:
            return f"Sorry, I couldn't generate a visualization for: '{query}'.\n{response}"
        else:
            return f"Here's the visualization code for '{query}':\n{response}"
    except Exception as e:
        return f"Error processing visualization query '{query}': {e}"
    finally:
        session.close()


In [12]:
executor,agent = create_agent_executor()

Database connection established!
2025-03-23 15:47:37,919 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-03-23 15:47:37,920 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-23 15:47:37,921 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-03-23 15:47:37,922 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-23 15:47:37,924 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-03-23 15:47:37,925 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-23 15:47:37,927 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-23 15:47:37,928 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `finquery`
2025-03-23 15:47:37,928 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-23 15:47:37,930 INFO sqlalchemy.engine.Engine ROLLBACK
2025-03-23 15:47:37,932 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-23 15:47:37,933 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `finquery`
2025-03-23 15:47:37,934 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-23 15:47:37,936

/Users/ericmelz/Data/code/finquery/venv/lib/python3.13/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
